In [39]:
import h5py 
import numpy as np
import os

Debido a la estructura del archivo, debería leerse dos veces para poder calcular desde 
antes las dimensiones finales de los arrays a cargar. Si el archivo es masivo (>500.000 entradas)
es importante evitar concatenar arrays (numpy) o tablas (pandas) dado que ese proceso demanda RAM.

Para comenzar, la rutina que lo explora y carga todas las dimensiones:

In [53]:
def get_datainfo(filename):
    """
    Rutina para leer la información (dimensiones totales) de un archivo NDskl_ascii
    """
    d = {}
    with open(filename, "r") as f:
        ## esto puede ser un problema si el archivo es muy grande
        #lines = f.readlines()
        ## Lo cambio por una lectura progresiva:

        # 1. Cargamos ndims para crear objetos de tamaño adecuado:
        _ = f.readline()
        d['Ndims'] = int(f.readline().split()[0])

        # 2. Nos saltamos los comentarios:
        line = f.readline()
        while '#' == line[0]: line = f.readline()

        # 3. Guardamos las dimensiones espaciales:
        if "BBOX" in line.split()[0]:
            d["BBOX_min"] = [float(i) for i in line.split()[1].replace("[","").replace("]","").split(",")]
            d["BBOX_max"] = [float(i) for i in line.split()[2].replace("[","").replace("]","").split(",")]

        # 4.1. Leemos la cantidad de puntos críticos:
        while "[CRITICAL POINTS]" not in line: line = f.readline()
        d["CriticalPoints"] = int(f.readline().split()[0])

        # 4.2 Por cada punto critico sumamos su cantidad de filamentos:
        totFilsinCP = 0
        for _ in range(d["CriticalPoints"]):
            _ = f.readline()
            nFilsinCP = int(f.readline().split()[0])
            totFilsinCP += nFilsinCP
            # Nos saltamos esas líneas:
            for _ in range(nFilsinCP): _ = f.readline()
        d["CriticalPoints_NumFils"] = totFilsinCP

        # 5.1 Ahora vamos al bloque de Filamentos, cargando total: 
        while "[FILAMENTS]" not in line: line = f.readline()
        d["Filaments"] = int(f.readline().split()[0])

        # 5.2 Por cada filamento, sumamos sus puntos conectores:
        totPointsinFil = 0
        for _ in range(d["Filaments"]):
            nPointsinFil = int(f.readline().split()[-1])
            totPointsinFil += nPointsinFil
            # Nos saltamos estas líneas:
            for _ in range(nPointsinFil): _ = f.readline()
        d["Filaments_NumPoints"] = totPointsinFil
                
        # 6.1 Ahora cargaremos la sección CRITICAL POINTS DATA
        # Esta es simplemente una Tabla con dims: N_CP x NF, donde
        # NF es el número de columnas (primera info que leeremos):
        while "[CRITICAL POINTS DATA]" not in line: line = f.readline()
        d["CriticalPointsData_NF"] = int(f.readline().split()[0])
        d["CriticalPointsData_Fields"] = []
        for _ in range(d["CriticalPointsData_NF"]): 
            d["CriticalPointsData_Fields"].append(f.readline().strip())

        # 6.2 Ahora que tenemos los NF y sus nombres, nos saltamos los registros:
        for _ in range(d["CriticalPoints"]): _ = f.readline()

        # 7.1 Ahora cargaremos la sección FILAMENTS DATA
        # Esta es simplemente una Tabla con dims: N_Fil x NF, donde
        # NF es el número de columnas de esta tabla (primera info que leeremos):
        while "[FILAMENTS DATA]" not in line: line = f.readline()
        d["Filaments_NF"] = int(f.readline().split()[0])
        d["Filaments_Fields"] = []
        for _ in range(d["Filaments_NF"]): 
            d["Filaments_Fields"].append(f.readline().strip())

        # 7.2 Ahora que tenemos los NF y sus nombres, nos saltamos los registros:
        # Ojo que esta tabla tiene todos los puntos conectores de todos los filamentos, en orden:
        for _ in range(d["Filaments_NumPoints"]): _ = f.readline()
        # Done! 
    return d

Ahora, utilizando la misma anterior como punto de partida, completaremos las seccionas que nos saltamos la primera vez e iremos guardando en arrays y tablas. 
Para hacer este código modular (y extensible) desde el inicio, agregaremos un parámetro opcional que permita seleccionar qué datos del archivo se quiere cargar, y condicionaremos cada parte a estos tags.

In [50]:
def read_NDskl_ascii(filename, fields=["[CRITICAL POINTS]","[FILAMENTS]","[CRITICAL POINTS DATA]", "[FILAMENTS DATA]"]):
    """
    Rutina para leer la información (dimensiones totales) de un archivo NDskl_ascii

    Params:
        - fields (opcional): Permite elegir qué campos del archivo se quieren cargar (todos por defecto).
                             Recibe una lista donde cada elemento almacena el nombre del campo deseado.
    """
    # Primero obtenemos la informacion del archivo:
    ndinfo = get_datainfo(filename)

    # Armamos la estrutura del contenedor
    d = {"METADATA":ndinfo}
    d["CRITICAL_POINTS"] = {}
    d["FILAMENTS"] = {} 
    d["CRITICAL_POINTS_DATA"] = None
    d["FILAMENTS_DATA"] = None

    # Ahora leemos cada parte 
    with open(filename, "r") as f:

        Ndims = ndinfo["Ndims"]

        # Leemos hasta el primer bloque:
        while "[CRITICAL POINTS]" not in line: line = f.readline()
        if "[CRITICAL POINTS]" in fields:
            # Creamos nuestros arrays:
            # FIXME: Como cada punto comienza con una linea con su info, 
            # meteré todo en arrays de nympy porque no sé como armarlo 
            # a saltos por registro en pandas...
            d["CRITICAL_POINTS"]["CPinfo"] = {}
            fd = d["CRITICAL_POINTS"]["CPinfo"]
            numCP = ndinfo["CriticalPoints"]
            fd["type"] = np.empty((numCP,), dtype=np.int16)
            fd["Pos_0"] = np.empty((numCP,), dtype=np.float)
            fd["Pos_1"] = np.empty((numCP,), dtype=np.float)
            fd["value"] = np.empty((numCP,), dtype=np.float)
            fd["pairID"] = np.empty((numCP,), dtype=np.int32)
            fd["boundary"] = np.empty((numCP,), dtype=np.int16)
            # Valores que almacenamos para encontrarlos en la tabla:
            fd["pairI"] = np.empty((numCP,), dtype=np.int32)
            fd["pairID"] = np.empty((numCP,), dtype=np.int32)
            

            ## ACA VOY!!!!
            return
            
            d["CriticalPoints"] = int(f.readline().split()[0])

            totFilsinCP = 0
            for _ in range(d["CriticalPoints"]):
                _ = f.readline()
                nFilsinCP = int(f.readline().split()[0])
                totFilsinCP += nFilsinCP
                # Nos saltamos esas líneas:
                for _ in range(nFilsinCP): _ = f.readline()
            d["CriticalPoints_NumFils"] = totFilsinCP

        # 5.1 Ahora vamos al bloque de Filamentos, cargando total: 
        while "[FILAMENTS]" not in line: line = f.readline()
        d["Filaments"] = int(f.readline().split()[0])

        # 5.2 Por cada filamento, sumamos sus puntos conectores:
        totPointsinFil = 0
        for _ in range(d["Filaments"]):
            nPointsinFil = int(f.readline().split()[-1])
            totPointsinFil += nPointsinFil
            # Nos saltamos estas líneas:
            for _ in range(nPointsinFil): _ = f.readline()
        d["Filaments_NumPoints"] = totPointsinFil
                
        # 6.1 Ahora cargaremos la sección CRITICAL POINTS DATA
        # Esta es simplemente una Tabla con dims: N_CP x NF, donde
        # NF es el número de columnas (primera info que leeremos):
        while "[CRITICAL POINTS DATA]" not in line: line = f.readline()
        d["CriticalPointsData_NF"] = int(f.readline().split()[0])
        d["CriticalPointsData_Fields"] = []
        for _ in range(d["CriticalPointsData_NF"]): 
            d["CriticalPointsData_Fields"].append(f.readline().strip())

        # 6.2 Ahora que tenemos los NF y sus nombres, nos saltamos los registros:
        for _ in range(d["CriticalPoints"]): _ = f.readline()

        # 7.1 Ahora cargaremos la sección FILAMENTS DATA
        # Esta es simplemente una Tabla con dims: N_Fil x NF, donde
        # NF es el número de columnas de esta tabla (primera info que leeremos):
        while "[FILAMENTS DATA]" not in line: line = f.readline()
        d["Filaments_NF"] = int(f.readline().split()[0])
        d["Filaments_Fields"] = []
        for _ in range(d["Filaments_NF"]): 
            d["Filaments_Fields"].append(f.readline().strip())

        # 7.2 Ahora que tenemos los NF y sus nombres, nos saltamos los registros:
        # Ojo que esta tabla tiene todos los puntos conectores de todos los filamentos, en orden:
        for _ in range(d["Filaments_NumPoints"]): _ = f.readline()
        # Done! 
    return d

In [49]:
### Codigo para testear la rutina:

filename = "simu_2D.ND.NDnet_s3_ascii"

d = get_datainfo(filename)
print(d)


{'Ndims': 2, 'BBOX_min': [0.0, 0.0], 'BBOX_max': [50000.0, 50000.0], 'CriticalPoints': 2116, 'CriticalPoints_NumFils': 4232, 'Filaments': 2116, 'Filaments_NumPoints': 71703, 'CriticalPointsData_NF': 9, 'CriticalPointsData_Fields': ['persistence_ratio', 'persistence_nsigmas', 'persistence', 'persistence_pair', 'parent_index', 'parent_log_index', 'log_field_value', 'field_value', 'cell'], 'Filaments_NF': 5, 'Filaments_Fields': ['field_value', 'orientation', 'cell', 'log_field_value', 'type']}


Acá desarrollé la _python-magick_ de la lectura de la dimensiones en ascii ;-)  

In [28]:
# esto se lee del archivo:
a,b = "[0,0]", "[5000,5000]"

In [30]:
# transformación a lista:
[float(i) for i in b.replace("[","").replace("]","").split(",")]

[5000.0, 5000.0]

###  Código incial trabajado en la reunión
(para comparar luego. Este extraía la sección de filamentos)

In [31]:


def extract_filaments(filename):
    """
    Rutina para extraer el bloque 1 de filamentos de un archivo NDskl_ascii
    """
    d = {}
    with open(filename, "r") as f:
        with open("filaments.dat", "w") as fout:
            ## esto puede ser un problema si el archivo es muy grande
            #lines = f.readlines()
            ## Lo cambio por una lectura progresiva:

            # 1. Cargamos ndims para crear objetos de tamaño adecuado:
            _ = f.readline()
            d['Ndims'] = int(f.readline().spli()[0])

            # 2. Nos saltamos los comentarios:
            line = f.readline()
            if '#' == line[0]: line = f.readline()

            # 3. Guardamos las dimensiones espaciales:
            if "BBOX" in line.split()[0]:
                
            
            # Nos saltamos todas las líneas hasta encontrar 
            # el tag del bloque que no interesa:
            while True:
                df = {}
                line = f.readline()
                if "[FILAMENTS]" in line:
                    df["findex"] = i
                    df["nFilaments"] = int(f.readline().spli()[0])
                    break
                    

                    
            currentfil = findex+1
            totallines = 0
            print(f"{currentfil} {totallines}")
            for i in range(2):
                totallines += lines[currentfil].split()[2]+1
                currentfil += lines[currentfil].split()[2]+1

            for i in range(totallines):
                fout.write(lines[i+findex])        

IndentationError: expected an indented block after 'if' statement on line 21 (53757026.py, line 26)

In [16]:
extract_filaments(filename)

8471 0


IndexError: list index out of range